# Import Files & Libs

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount("/content/gdrive", force_remount=True)
!cp /content/gdrive/MyDrive/Decathlon/project.zip /content/
!unzip -q /content/project.zip
!rm /content/project.zip
!mkdir /content/results

In [59]:
import torch
import torch.nn
import numpy as np
import matplotlib.pyplot as plt

from model import lstm
from dataset import dataset
from training import training


import torch.optim as optim
import tqdm
import os
from importlib import reload
reload(lstm)
reload(dataset)
reload(training)

<module 'training.training' from 'c:\\Users\\Edouard\\github\\DecathlonTest\\training\\training.py'>

# Load Data

In [41]:
ds_train=dataset.Turnover_dataset()
ds_val=dataset.Turnover_dataset()
with open('/content/train_data.json', 'r') as f:
    ds_train.load_from_json(f)
with open('/content/val_data.json', 'r') as f:
    ds_val.load_from_json(f)
ds_train.set_data_for_training(False)
ds_val.set_data_for_training(False)
dataloader_t = torch.utils.data.DataLoader(ds_train,batch_size=32,shuffle=True,drop_last=False) #Dataloader train set
dataloader_v = torch.utils.data.DataLoader(ds_val,batch_size=32,shuffle=True,drop_last=False) #Dataloader validation set

# Model

In [60]:
device= torch.device("cuda:0")
model = lstm.LSTM_Turnoverv2(True,hidden_size=48).to(device)

In [45]:
param=0
for key in model.state_dict():
    param+=model.state_dict()[key].view(-1).shape[0]
print(param)

30455


In [13]:
optimizer = optim.Adam(model.parameters(), lr=3e-4, betas=(0.9, 0.999)) #Adam Optimizer 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.3,verbose=True) #Learning rate scheduler
num_epochs=20

Adjusting learning rate of group 0 to 3.0000e-04.


In [ ]:
training.training(num_epochs,model,optimizer,scheduler,dataloader_t,dataloader_v,device,"/content/results/")

In [ ]:
model.load_state_dict(torch.load("/content/results/checkpoint_14.pth")['model_state_dict'])
data=next(iter(dataloader_v))
future_pred=data["y"].shape[1]
y=model(data["x"].float().to(device),data["dep"].float().to(device),data["idr"].float().to(device),
    data["zod"].float().to(device),data["add_input"].float().to(device),future_pred)
y=y[:,-future_pred:]
gt=data["y"].float()
pred=y[0].to(torch.device("cpu"))
g=gt[0]
id=data["id"][0]
sample=ds_val.samples[id]
raw_xy=sample["raw_xy"]
min=sample["min_x"]
max=sample["max_x"]
diff_recons=np.concatenate((dataset.inversenorm(data["x"][0].float().numpy(),max,min),dataset.inversenorm(pred.detach().numpy(),max,min)))
final=dataset.inverse_diff(raw_xy[0],diff_recons)
plt.plot(final,color='red')
plt.plot(raw_xy)